In [ ]:
from fireworks import FireworksClient

# Create a client instance
client = FireworksClient()

# Make a request to the "llama2 7b" chat
response = client.request_chat("llama2 7b")

# Print the response
print(response)


In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [4]:
import pathlib
import textwrap

import google.generativeai as genai

genai.configure(api_key=GOOGLE_API_KEY)

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

/home/vpmb/Documents/python/dataset-generation/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-pro
models/gemini-pro-vision


In [6]:
model = genai.GenerativeModel('gemini-pro')

In [85]:
%%time

def generate_response(content:str):
    response = model.generate_content(f"""
{SYSTEM_PROMPT}+{content}
""")
    return response

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 4.77 µs


In [86]:
response = generate_response(CONTEXT)

In [89]:
SYSTEM_PROMPT = """
You are supposed to create  a dataset so refrain from outputting anything unnecessary .
You are given pages of lecture slides from a curated collection. Ignore page numbers and personal information, extract the context.
Generate  logical questions, correct answer and three incorrect answers suitable for mcq from the context text. 
Do not generate questions that go like from the lecture? in this lecture? etc.
Go step by step.  final output should be in json format. json schema for the output is 
{
    "Output": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string"
            },
            "correct_answer": {
                "type": "string"
            },
            "incorrect_answers": {
                "type": "array",
                "items": {
                    "type": "string"
                }
            }
        },
        "required": ["question", "correct_answer", "incorrect_answers"]
    }
}


"""

In [82]:
CONTEXT= """
Networked computers
■Computers connected to each other
–Wired connections using cables
–Wireless using WIFI, 4G/5G 
■We will concentrate on wired arrangement
4 Computer Systems Fundamentals - 4COSC004W
"""

In [87]:
to_markdown(response.text)

> {
>  "Output": {
>   "question": "What type of connections are used to connect computers to each other?",
>   "correct_answer": "Wired connections using cables or Wireless using WIFI, 4G/5G",
>   "incorrect_answers": [
>    "Bluetooth",
>    "Infrared",
>    "All of the above"
>   ]
>  }
> }

In [41]:
folder_path = './dataset/source/'
pdfs = os.listdir(folder_path)
pdfs

['lecture 2 Part B 2020.pdf',
 'Week09 - Tagged.pdf',
 'Week07.pdf',
 'Week04_lecture.pdf',
 'Week03.pdf',
 'PBS  Lecture (1).pdf',
 'for_loop.pdf',
 'Lists_Tuples_20_21.pdf',
 'ListComprehensions21-22.pdf',
 'functions_21-22_v2.pdf',
 'CSF_Lecture_1_Part_1 of 2.pdf',
 'Week05(1) - Tagged.pdf',
 'string slices.pdf',
 'Week02 (1).pdf',
 'Range of Values (Unsigned)(1).pdf',
 '3 Images.pdf',
 '2 Character Codes .pdf',
 'continue_break.pdf',
 'Week10 - Tagged.pdf',
 'DiskSector 0.pdf',
 '1 Data Storage.pdf',
 'Binary Operations.pdf',
 'RANGE_FUNCTION_21_22.pdf',
 'stringFormatting.pdf',
 'Week08 - Tagged.pdf',
 'while_loops.pdf',
 'CSF_Lecture_Operating Systems.pdf',
 'strings.pdf',
 'Signed Integers.pdf',
 'CSF_IP Addressing.pdf',
 'CSF_Lecture_1_Part_2 of 2.pdf',
 'files_21-22.pdf',
 'CSF_Lecture2.pdf',
 'Dictionary.pdf',
 'python-sqlite_21-21_V2.pdf',
 'conditionsWeek3_20_21_v1.pdf',
 'lecture 2 Part A 2020.pdf']

In [44]:

import PyPDF2
import pandas as pd

def pdf_to_pandas_df(folder_path, pdfs):
    all_pdfs = []
    for pdf in pdfs:
        with open(folder_path+pdf, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            data = {'Page':[page.extract_text() for page in reader.pages]}
            df = pd.DataFrame(data)
            all_pdfs.append(df)
    df = pd.concat(all_pdfs, ignore_index=True)
    return df


# Example usage
df = pdf_to_pandas_df(folder_path, pdfs)
# print(pages)


In [100]:
df = pd.read_csv('./dataset/filtered_dataset.csv')

In [99]:
df

,0
0,"{\n ""Output"": {\n ""question"": ""Which..."


In [105]:
import time

import pandas as pd

def process_row(row):
    response = generate_response(row[0])
    time.sleep(1)
    return response.text

def gen_dataset(df):
    df['Output'] = df.apply(process_row, axis=1)
    return df

df1 = gen_dataset(df)

/tmp/ipykernel_11681/2204452987.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  response = generate_response(row[0])


In [106]:
df1

,"Summary\n•Collections and Data Structure \n•Arrays\n•List, Queue, Map\n•Searching and Sorting\n•Linked List",102,Output
0,11Lecture Self-Check Question \nAssign a varia...,104.0,"{\n ""Output"": {\n ""question"": ""Where..."
1,•Give the program \nadditional conditions. \n•...,105.0,"{\n ""Output"": {\n ""question"": ""What type of r..."
2,•Java Packages•The String Class•Using System.o...,106.0,"{\n ""Output"": {\n ""question"": ""Which..."
3,TYPES OF NETWORK\nPeer-to-peer\nClient-Server\...,106.0,"{\n ""Output"": {\n ""question"": ""Which..."
4,Physical Topologies\n6Bus\nRing\nStarExtended ...,107.0,"{\n ""Output"": {\n ""question"": ""Which..."
...,...,...,...
195,How can we improve our code?\n•Validation : \n...,215.0,"{\n ""Output"": {\n ""question"": ""What ..."
196,By the end of this unit you will:\n■Understand...,215.0,"{\n ""Output"": {\n ""question"": ""What ..."
197,File Class\n•Useful for retrieving information...,215.0,"{\n ""Output"": {\n ""question"": ""What ..."
198,Microcomputer\n■OS needs to:\n–Initialise the ...,215.0,"{\n ""Output"": {\n ""question"": ""Accor..."


In [107]:
df2 = pd.read_csv('./dataset/modified_dataset.csv')

In [108]:
df2.head()

,Source,Output
0,11Lecture Self-Check Question \nAssign a varia...,"{\n ""Output"": {\n ""question"": ""Where..."
1,•Give the program \nadditional conditions. \n•...,"{\n ""Output"": {\n ""question"": ""What type of r..."
2,•Java Packages•The String Class•Using System.o...,"{\n ""Output"": {\n ""question"": ""Which..."
3,TYPES OF NETWORK\nPeer-to-peer\nClient-Server\...,"{\n ""Output"": {\n ""question"": ""Which..."
4,Physical Topologies\n6Bus\nRing\nStarExtended ...,"{\n ""Output"": {\n ""question"": ""Which..."
